In [21]:
import pandas as pd
import numpy as np

raw = pd.read_csv("JPM-15Oct25.csv")
raw = raw[raw["Type"].isin(["Trade", "Quote"])].copy()

raw["DT"] = pd.to_datetime(raw["Date-Time"])
raw = raw.sort_values("DT").reset_index(drop=True)


In [23]:
q = raw[
    (raw["Type"] == "Quote") &
    raw["Bid Price"].notna() &
    raw["Ask Price"].notna() &
    (raw["Ask Price"] > raw["Bid Price"])
].copy()

quotes = (
    q.groupby(["#RIC", "DT"])
     .agg(
         BID=("Bid Price", "max"),
         OFR=("Ask Price", "min"),
         BIDSIZ=("Bid Size", "sum"),
         OFRSIZ=("Ask Size", "sum"),
     )
     .reset_index()
     .rename(columns={"#RIC": "SYMBOL"})
)


In [25]:
def safe_vwap(prices, volumes):
    v = volumes.values
    if v.sum() == 0:
        return prices.mean()
    return np.average(prices, weights=v)


In [26]:
t = raw[
    (raw["Type"] == "Trade") &
    raw["Price"].notna() &
    raw["Volume"].notna()
].copy()

trades = (
    t.groupby(["#RIC", "DT"])
     .agg(
         PRICE=("Price", lambda x: safe_vwap(x, t.loc[x.index, "Volume"])),
         SIZE=("Volume", "sum"),
         NUMTRADES=("Volume", "count"),
         EX=("Ex/Cntrb.ID", lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
     )
     .reset_index()
     .rename(columns={"#RIC": "SYMBOL"})
)


In [27]:
TAQ = pd.merge_asof(
    trades.sort_values(["SYMBOL", "DT"]),
    quotes.sort_values(["SYMBOL", "DT"]),
    on="DT",
    by="SYMBOL",
    direction="backward",
    tolerance=pd.Timedelta("1s"),
)

TAQ = TAQ.dropna(subset=["BID", "OFR"])


In [28]:
TAQ["MIDQUOTE"] = (TAQ["BID"] + TAQ["OFR"]) / 2

TAQ = TAQ[
    ["DT", "SYMBOL", "BID", "OFR", "OFRSIZ", "BIDSIZ",
     "MIDQUOTE", "PRICE", "NUMTRADES", "SIZE", "EX"]
].reset_index(drop=True)


In [29]:
TAQ.head()

,DT,SYMBOL,BID,OFR,OFRSIZ,BIDSIZ,MIDQUOTE,PRICE,NUMTRADES,SIZE,EX
0,2025-10-15 04:00:00.025232297-04:00,JPM,301.95,302.72,2.0,2.0,302.335,302.72,1,10.0,DEX
1,2025-10-15 04:00:00.040235123-04:00,JPM,304.00,304.35,2.0,4.0,304.175,304.50,1,10.0,PSE
2,2025-10-15 04:00:00.095510462-04:00,JPM,304.00,304.35,2.0,4.0,304.175,303.36,2,180.0,DEX
3,2025-10-15 04:00:00.104908888-04:00,JPM,304.00,304.35,2.0,4.0,304.175,304.00,1,3.0,PSE
4,2025-10-15 04:00:00.220395322-04:00,JPM,304.00,305.00,1.0,2.0,304.500,304.00,1,4.0,PSE


In [16]:
#   # change name if yours is slightly different

# # 2. Keep only trades & quotes
# raw = raw[raw["Type"].isin(["Trade", "Quote"])].copy()

# # 3. Parse timestamp and sort
# raw["DT"] = pd.to_datetime(raw["Date-Time"])
# raw = raw.sort_values("DT").reset_index(drop=True)

# # 4. Build TRADES table ---------------------------------------------
# trades = raw[
#     (raw["Type"] == "Trade") &
#     raw["Price"].notna() &
#     raw["Volume"].notna() &
#     (raw["Price"] > 0) &
#     (raw["Volume"] > 0)
# ].copy()

# trades = trades[["DT", "#RIC", "Price", "Volume", "Ex/Cntrb.ID"]]
# trades.columns = ["DT", "SYMBOL", "PRICE", "SIZE", "EX"]
# trades = trades.sort_values(["SYMBOL", "DT"]).reset_index(drop=True)

# # 5. Build QUOTES table ---------------------------------------------
# quotes = raw[
#     (raw["Type"] == "Quote") &
#     raw["Bid Price"].notna() &
#     raw["Ask Price"].notna() &
#     (raw["Bid Price"] > 0) &
#     (raw["Ask Price"] > raw["Bid Price"])
# ].copy()

# quotes = quotes[
#     ["DT", "#RIC", "Bid Price", "Bid Size", "Ask Price", "Ask Size", "Ex/Cntrb.ID"]
# ]
# quotes.columns = ["DT", "SYMBOL", "BID", "BIDSIZ", "OFR", "OFRSIZ", "EX"]
# quotes = quotes.sort_values(["SYMBOL", "DT"]).reset_index(drop=True)

# # 6. Match trades to most recent quote (TAQ) ------------------------
# TAQ = pd.merge_asof(
#     trades.sort_values(["SYMBOL", "DT"]),
#     quotes.sort_values(["SYMBOL", "DT"]),
#     on="DT",
#     by="SYMBOL",
#     direction="backward",
#     tolerance=pd.Timedelta("1s"),
# )

# # Drop trades with no valid quote match
# TAQ = TAQ.dropna(subset=["BID", "OFR"]).reset_index(drop=True)

# # ✅ Fix exchange columns
# TAQ = TAQ.rename(columns={
#     "EX_x": "TRD_EX",
#     "EX_y": "QTE_EX"
# })

# # 7. Add midquote + spread -----------------------------------------
# TAQ["MIDQUOTE"] = (TAQ["BID"] + TAQ["OFR"]) / 2
# TAQ["SPREAD"] = TAQ["OFR"] - TAQ["BID"]

# # Quick sanity check
# print(trades.shape, quotes.shape, TAQ.shape)
# TAQ.head()


(188000, 5) (89710, 7) (159831, 12)


,DT,SYMBOL,PRICE,SIZE,TRD_EX,BID,BIDSIZ,OFR,OFRSIZ,QTE_EX,MIDQUOTE,SPREAD
0,2025-10-15 04:00:00.025232297-04:00,JPM,302.72,10.0,DEX,301.95,1.0,304.35,1.0,NaN,303.150,2.40
1,2025-10-15 04:00:00.040235123-04:00,JPM,304.50,10.0,PSE,304.00,3.0,304.35,1.0,NaN,304.175,0.35
2,2025-10-15 04:00:00.095510462-04:00,JPM,302.72,90.0,DEX,304.00,2.0,304.35,1.0,NaN,304.175,0.35
3,2025-10-15 04:00:00.095510462-04:00,JPM,304.00,90.0,PSE,304.00,2.0,304.35,1.0,NaN,304.175,0.35
4,2025-10-15 04:00:00.104908888-04:00,JPM,304.00,3.0,PSE,304.00,2.0,304.35,1.0,NaN,304.175,0.35


# Task
Export the `TAQ` DataFrame to a CSV file named "TAQ_data.csv" and then provide the R code to import "TAQ_data.csv" into an R DataFrame.

## Export TAQ to CSV

### Subtask:
Export the `TAQ` DataFrame to a CSV file named 'TAQ_data.csv' for easy import into R.


**Reasoning**:
The subtask requires exporting the `TAQ` DataFrame to a CSV file named 'TAQ_data.csv' without the index. I will use the `to_csv()` method with `index=False`.



In [30]:
TAQ.to_csv('TAQ_data.csv', index=False)
print("TAQ DataFrame exported to 'TAQ_data.csv'")

TAQ DataFrame exported to 'TAQ_data.csv'


```R
# Import the CSV file into an R DataFrame
TAQ_r <- read.csv('TAQ_data.csv')

# Display the first few rows of the R DataFrame
head(TAQ_r)

# Display the structure of the R DataFrame to verify column types
str(TAQ_r)
```

```R
# Import the CSV file into an R DataFrame
TAQ_r <- read.csv('TAQ_data.csv')

# Display the first few rows of the R DataFrame
head(TAQ_r)

# Display the structure of the R DataFrame to verify column types
str(TAQ_r)
```

```R
# Import the CSV file into an R DataFrame
TAQ_r <- read.csv('TAQ_data.csv')

# Display the first few rows of the R DataFrame
head(TAQ_r)

# Display the structure of the R DataFrame to verify column types
str(TAQ_r)
```

```R
# Import the CSV file into an R DataFrame
TAQ_r <- read.csv('TAQ_data.csv')

# Display the first few rows of the R DataFrame
head(TAQ_r)

# Display the structure of the R DataFrame to verify column types
str(TAQ_r)
```

## Final Task

### Subtask:
Confirm that the TAQ data has been successfully exported and instructions for R import have been provided.


## Summary:

### Q&A
1.  **Was the TAQ data successfully exported?**
    Yes, the `TAQ` DataFrame was successfully exported to a CSV file named "TAQ_data.csv".
2.  **Were instructions for R import provided?**
    Yes, R code to import "TAQ_data.csv" into an R DataFrame was provided, including commands to read the CSV, display the first few rows, and check the structure.

### Data Analysis Key Findings
*   The `TAQ` DataFrame was successfully exported to a CSV file named "TAQ_data.csv" using the `to_csv()` method, ensuring the DataFrame's index was not included as a column.
*   Instructions for importing the "TAQ_data.csv" file into an R DataFrame were provided. The R code snippet includes `read.csv('TAQ_data.csv')` for importing, `head(TAQ_r)` to inspect the initial rows, and `str(TAQ_r)` to verify the data structure and column types in R.

### Insights or Next Steps
*   Verify the integrity and correctness of the data after importing it into R to ensure all data types and values are preserved as expected.
*   Proceed with the planned analysis within the R environment, utilizing the successfully imported `TAQ_r` DataFrame.
